In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import datetime as datetime
import time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains 

option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
#actions = ActionChains(driver)
#link equal to daily covid data 
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratecases"
driver.get(link)
#let driver get link, takes time to load everything in
time.sleep(10)
#click on drop down menu and first option in drop down menu for states 
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/div[2]/div[2]").click()
#parse through the html
source = driver.page_source 
#information needed is embedded within several tags so we go through each tag to get the info we want
soup = BeautifulSoup(source, 'html.parser')
soup_2 =  BeautifulSoup(source, 'html.parser')
info = soup.find_all('table', class_= 'expanded-data-table')
info_2 = soup.find_all('tr')
info_3 = soup.find_all('td')
info_3 = list(info_3)

#scrape the list of dates available in the data
for i in range(0,len(info_3)):
    info_3[i] = info_3[i]['aria-label']
date = []
for i in range(0,len(info_3),3):
    holder = info_3[i].split(' ')
    date.append(holder[1]+' ' + holder[2]+ ' '+ holder[3])

#create dataframe for everything scrapped starting with Date as the first column
covid_data = list(zip(date))
covid_data = pd.DataFrame(covid_data, columns = ['Date'])

#scrape all the state names as the CDC data includes NYC as separate as well as territories
state_names = []
states = soup_2.find_all('div', class_ = 'item')
for i in range(0,len(states)):
    state_names.append(states[i].text)

#chromedriver to go to the cdc website 
option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratecases"
driver.get(link)
time.sleep(10)
#click on the dropdown menu again
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
time.sleep(5)
#get total count of number of items in dropdown menu
total_count = driver.find_elements_by_xpath("//div[@class='menu transition visible']/div")
#loop through drop down menu one by one
#reason for this is that the table on the site is updated as you click through each option
#each option is clicked, the data of interest is scrapped, and next item is clicked and so on
for i in range(len(total_count)):

    path = "//div[@class='menu transition visible']/div[{}]".format(i+1)

    driver.find_element_by_xpath(path).click()
    ####scrape here
    source = driver.page_source 
    soup = BeautifulSoup(source, 'html.parser')
    info = soup.find_all('table', class_= 'expanded-data-table')
    info_2 = soup.find_all('tr')
    info_3 = soup.find_all('td')
    info_3 = list(info_3)
    #loop through to remove the aria-label in front of each piece of data we want
    for x in range(0,len(info_3)):
        info_3[x] = info_3[x]['aria-label']
    daily_case = []
    case_rate = []

    #find the daily cases and well as cases per 100,000
    for x in range(1, len(info_3),3):
        holder = info_3[x].split(' ')
        daily_case.append(holder[-1])

    for x in range(2, len(info_3),3):
        holder = info_3[x].split(' ')
        case_rate.append(holder[-1])
    #scrapped data added to overall dataframe including name of territory/state as well as what the data represents
    
    covid_data[state_names[i] + ' Total Cases'] = list(daily_case)
    covid_data[state_names[i] + ' Cases Per 100,000'] = list(case_rate)
    print(state_names[i]+' read in for cases')
    #click drop down menu to reset 
    driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
    time.sleep(3)
    
#below is the same process but for COVID death data whereas above was just for cases
option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
#actions = ActionChains(driver)
#link equal to daily covid data 
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratedeaths"
driver.get(link)
#let driver get link, takes time to load everything in
time.sleep(10)

driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/div[2]/div[2]").click()
#parse through the html
source = driver.page_source 
#information needed is embedded within several tags so we go through each tag to get the info we want
soup = BeautifulSoup(source, 'html.parser')
soup_2 =  BeautifulSoup(source, 'html.parser')
info = soup.find_all('table', class_= 'expanded-data-table')
info_2 = soup.find_all('tr')
info_3 = soup.find_all('td')
info_3 = list(info_3)

for i in range(0,len(info_3)):
    info_3[i] = info_3[i]['aria-label']
date = []
for i in range(0,len(info_3),3):
    holder = info_3[i].split(' ')
    date.append(holder[1]+' ' + holder[2]+ ' '+ holder[3])

covid_data_death = list(zip(date))
covid_data_death = pd.DataFrame(covid_data_death, columns = ['Date'])
    
state_names = []
states = soup_2.find_all('div', class_ = 'item')
for i in range(0,len(states)):
    state_names.append(states[i].text)

option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(),options = option)
link = "https://covid.cdc.gov/covid-data-tracker/#trends_totalandratedeaths"
driver.get(link)
time.sleep(10)
driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
time.sleep(5)
total_count = driver.find_elements_by_xpath("//div[@class='menu transition visible']/div")
for i in range(len(total_count)):
   
    path = "//div[@class='menu transition visible']/div[{}]".format(i+1)

    driver.find_element_by_xpath(path).click()
    ####scrape here
    source = driver.page_source 
    soup = BeautifulSoup(source, 'html.parser')
    info = soup.find_all('table', class_= 'expanded-data-table')
    info_2 = soup.find_all('tr')
    info_3 = soup.find_all('td')
    info_3 = list(info_3)
    #loop through to remove the aria-label in front of each piece of data we want
    for x in range(0,len(info_3)):
        info_3[x] = info_3[x]['aria-label']
    daily_case = []
    case_rate = []

   #get daily deaths as well as deaths per 100,000
    for x in range(1, len(info_3),3):
        holder = info_3[x].split(' ')
        daily_case.append(holder[-1])

    for x in range(2, len(info_3),3):
        holder = info_3[x].split(' ')
        case_rate.append(holder[-1])

    covid_data_death[state_names[i] + ' Total Deaths'] = list(daily_case)
    covid_data_death[state_names[i] + ' Deaths Per 100,000'] = list(case_rate)
    print(state_names[i]+' read in for deaths')
    driver.find_element_by_xpath("/html/body/main/div/div[1]/div/div/div[1]/div[1]/div/div[1]/div[2]/i").click()
    time.sleep(3)
covid_data.to_csv('covid_state_cases.csv')
covid_data_death.to_csv('covid_state_deaths.csv')

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\anson\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\anson\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
United States read in for cases
Alabama read in for cases
Alaska read in for cases
Arizona read in for cases
Arkansas read in for cases
California read in for cases
Colorado read in for cases
Commonwealth of the Northern Mariana Islands read in for cases
Connecticut read in for cases
Delaware read in for cases
District of Columbia read in for cases
Federated States of Micronesia read in for cases
Florida read in for cases
Georgia read in for cases
Guam read in for cases
Hawaii read in for cases
Idaho read in for cases
Illinois read in for cases
Indiana read in for cases
Iowa read in for cases
Kansas read in for cases
Kentucky read in for cases
Louisiana read in for cases
Maine read in for cases
Maryland read in for cases
Massachusetts read in for cases
Michigan read in for cases
Minnesota read in for cases
Mississippi read in for cases
Missouri read in for cases
Montana read in for cases
Nebraska read in for cases
Nevada read in for cases
New Hampshire read in for cases
New Jersey re

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\anson\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\anson\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
United States read in for deaths
Alabama read in for deaths
Alaska read in for deaths
Arizona read in for deaths
Arkansas read in for deaths
California read in for deaths
Colorado read in for deaths
Commonwealth of the Northern Mariana Islands read in for deaths
Connecticut read in for deaths
Delaware read in for deaths
District of Columbia read in for deaths
Federated States of Micronesia read in for deaths
Florida read in for deaths
Georgia read in for deaths
Guam read in for deaths
Hawaii read in for deaths
Idaho read in for deaths
Illinois read in for deaths
Indiana read in for deaths
Iowa read in for deaths
Kansas read in for deaths
Kentucky read in for deaths
Louisiana read in for deaths
Maine read in for deaths
Maryland read in for deaths
Massachusetts read in for deaths
Michigan read in for deaths
Minnesota read in for deaths
Mississippi read in for deaths
Missouri read in for deaths
Montana read in for deaths
Nebraska read in for deaths
Nevada read in for deaths
New Hampshir

In [157]:
covid_data

,Date,United States Total Cases,"United States Cases Per 100,000",Alabama Total Cases,"Alabama Cases Per 100,000",Alaska Total Cases,"Alaska Cases Per 100,000",Arizona Total Cases,"Arizona Cases Per 100,000",Arkansas Total Cases,...,Virginia Total Cases,"Virginia Cases Per 100,000",Washington Total Cases,"Washington Cases Per 100,000",West Virginia Total Cases,"West Virginia Cases Per 100,000",Wisconsin Total Cases,"Wisconsin Cases Per 100,000",Wyoming Total Cases,"Wyoming Cases Per 100,000"
0,Jan 6 2021,21259997,6486.2,384184,7835.4,47334,6470.4,574680,7895.3,242593,...,382679,4483.4,260752,3424.2,94678,5282.9,535999,9205.8,45890,7929
1,Jan 5 2021,20960093,6394.7,379593,7741.8,47006,6425.6,567474,7796.3,238888,...,377300,4420.4,258767,3398.2,93162,5198.3,531890,9135.2,45569,7873.6
2,Jan 4 2021,20732404,6325.2,374095,7629.6,46812,6399.1,561542,7714.8,234781,...,371913,4357.2,256435,3367.5,91886,5127.1,527871,9066.2,45247,7817.9
3,Jan 3 2021,20558489,6272.1,371934,7585.6,46552,6363.5,556384,7644,233475,...,367536,4306,255396,3353.9,91058,5080.9,526245,9038.2,44875,7753.7
4,Jan 2 2021,20346372,6207.4,369458,7535.1,46262,6323.9,539150,7407.2,231442,...,363765,4261.8,246752,3240.4,89327,4984.4,523652,8993.7,44573,7701.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,Jan 26 2020,5,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
347,Jan 25 2020,2,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
348,Jan 24 2020,2,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
349,Jan 23 2020,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [1]:
import pandas as pd
covid_data = pd.read_csv("covid_state_cases.csv") 
covid_data_death = pd.read_csv("covid_state_deaths.csv") 

In [30]:
variables = []
for column in covid_data.columns:
    variables.append(column)
total_cases = []
state_cases = []
for i in range(2,len(variables),2):
    total_cases.append(variables[i])
for i in range(3,len(variables),2):
    state_cases.append(variables[i])
    
    
variables = []
for column in covid_data_death.columns:
    variables.append(column)
total_deaths=[]
state_deaths = []
for i in range(2,len(variables),2):
    total_deaths.append(variables[i])
for i in  range(3,len(variables),2):
    state_deaths.append(variables[i])

In [38]:
state_cases_data = pd.DataFrame()
for i in range(len(state_cases)):
    state_cases_data[state_cases[i]] = covid_data[state_cases[i]]
state_total_cases_data = pd.DataFrame()
for i in range(len(total_cases)):
    state_total_cases_data[total_cases[i]] = covid_data[total_cases[i]]

state_deaths_data = pd.DataFrame()
for i in range(len(state_deaths)):
    state_deaths_data[state_deaths[i]] = covid_data_death[state_deaths[i]]
    
state_total_deaths_data = pd.DataFrame()
for i in range(len(total_deaths)):
    state_total_deaths_data[total_deaths[i]] = covid_data_death[total_deaths[i]]